# Comparação do Vader com outros algoritmos

In [9]:
from analise_sentimento_modelo import pondera_polaridade_titulo_texto, classifica_sentimento_vaderptbr
from vaderSentimentptbr import SentimentIntensityAnalyzer 
from sumarizador_textrankptbr import summarize_text_rank 
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd
import spacy
import nltk
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn import svm
import numpy as np
from unidecode import unidecode
from noticias_processamento_texto import lematizador
     

[nltk_data] Downloading package rslp to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Funçoes de avaliação

In [10]:
'''
Realiza a aplicação de cada algortimo no dataset, indicando suas métricas de Acurácia, Precisão, Cobertura e F1-measure
Para algortimos de machine learing, aplica 5-fold cross-validation
'''
def avalia_algoritmo(df, col_texto, col_titulo=None, col_sentimento='sentimento', sent_pos=1, sent_neu=0, sent_neg=-1, modelo='Vader', resumir=True):
    
    # vader - nao supervisionado
    
    if modelo == 'Vader':
        if col_titulo is not None:
            df['sentimento_vader'] = df.apply(lambda row: classifica_sentimento_vaderptbr(polaridade_sentimento_vaderptbr(row[col_texto], row[col_titulo], resumir), sent_pos, sent_neu, sent_neg) , axis=1)
        else:
            df['sentimento_vader'] = df.apply(lambda row: classifica_sentimento_vaderptbr(polaridade_sentimento_vaderptbr(row[col_texto], None, resumir), sent_pos, sent_neu, sent_neg) , axis=1)
        print('Accuracy Score: %.3f' % accuracy_score(df[col_sentimento], df['sentimento_vader']))
        print('Precision Score: %.3f' % precision_score(df[col_sentimento], df['sentimento_vader'], average='binary'))
        print('Recall Score: %.3f' % recall_score(df[col_sentimento], df['sentimento_vader'], average='binary'))
        print('F1 Score: %.3f' % f1_score(df[col_sentimento], df['sentimento_vader'], average='binary'))
        return 
    
    # machine learning com cross-validation
    
    df['texto_ajustado'] = df[col_texto].apply(str.lower)
    df['texto_ajustado'] = df['texto_ajustado'].apply(lematizador)

    X, y = df['texto_ajustado'], df[col_sentimento]
    metrics_acc = []
    metrics_p = []
    metrics_r = []
    metrics_f1 = []

    skf = StratifiedKFold(n_splits=5, shuffle=True)
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        vect = TfidfVectorizer(ngram_range=(1,2), max_features=1000 , stop_words=list(spacy.lang.pt.stop_words.STOP_WORDS) + ['abc', 'xyz', 'def'])
        X_train_dtm = vect.fit_transform(X_train)
        X_test_dtm = vect.transform(X_test)
        model=None
        if modelo =='MultinomialNB':
            model = MultinomialNB()
        elif modelo == 'LogisticRegression':
            model = LogisticRegression(max_iter=200)
        else:
            model = svm.SVC(kernel='linear')
        model.fit(X_train_dtm, y_train)
        y_pred_class = model.predict(X_test_dtm)

        metrics_acc.append(accuracy_score(y_test, y_pred_class))
        metrics_p.append(precision_score(y_test, y_pred_class, average='binary'))
        metrics_r.append(recall_score(y_test, y_pred_class, average='binary'))
        metrics_f1.append(f1_score(y_test, y_pred_class, average='binary'))

    metrics_acc = np.array(metrics_acc)
    metrics_p = np.array(metrics_p)
    metrics_r = np.array(metrics_r)
    metrics_f1 = np.array(metrics_f1)
    print('Media accuracy: ', np.mean(metrics_acc, axis=0))
    print('Media precision: ', np.mean(metrics_p, axis=0))
    print('Media recall: ', np.mean(metrics_r, axis=0))
    print('Media f1: ', np.mean(metrics_f1, axis=0))
    
    

# Comparação com dataset do trabalho de Magner et al. (2022)

In [11]:
print('Comparação com trabalho: Magner, Renata; Medeiros, Camila Alves; Felix, Klysman; Souza, Pedro Vieira. 2020. Análise de Sentimentos de Informações do Mercado Financeiro — Projeto de Conclusão de Curso — Data Science & Machine Learning da Tera. Disponível em: <https://medium.com/@renata.amp/análise-de-sentimentos-de-informações-do-mercado-financeiro-projeto-de-conclusão-de-curso-data-6a8f8dcbf441>\n')
df = pd.read_excel('datasets/Dataset_noticias_tera.xlsx')

print('\nAvaliando base Magner et al com VADER')
avalia_algoritmo(df, 'texto_noticia', col_titulo='titulo', col_sentimento='sentimento', sent_pos=1, sent_neu=1, sent_neg=0, modelo='Vader')

print('\nAvaliando base Magner et al com MultinomialNB')
avalia_algoritmo(df, 'texto_noticia', col_titulo='titulo', col_sentimento='sentimento', sent_pos=1, sent_neu=1, sent_neg=0, modelo='MultinomialNB')
    
print('\nAvaliando base Magner et al com LogisticRegression')
avalia_algoritmo(df, 'texto_noticia', col_titulo='titulo', col_sentimento='sentimento', sent_pos=1, sent_neu=1, sent_neg=0, modelo='LogisticRegression')

print('\nAvaliando base Magner et al com SVM')
avalia_algoritmo(df, 'texto_noticia', col_titulo='titulo', col_sentimento='sentimento', sent_pos=1, sent_neu=1, sent_neg=0, modelo='SVC')

Comparação com trabalho: Magner, Renata; Medeiros, Camila Alves; Felix, Klysman; Souza, Pedro Vieira. 2020. Análise de Sentimentos de Informações do Mercado Financeiro — Projeto de Conclusão de Curso — Data Science & Machine Learning da Tera. Disponível em: <https://medium.com/@renata.amp/análise-de-sentimentos-de-informações-do-mercado-financeiro-projeto-de-conclusão-de-curso-data-6a8f8dcbf441>


Avaliando base Magner et al com VADER


/Users/danielsaraivaleite/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Accuracy Score: 0.702
Precision Score: 0.736
Recall Score: 0.852
F1 Score: 0.790

Avaliando base Magner et al com MultinomialNB
Media accuracy:  0.7018388106416276
Media precision:  0.7099536663167245
Media recall:  0.9236702127659575
Media f1:  0.8026369226369227

Avaliando base Magner et al com LogisticRegression
Media accuracy:  0.7157668231611893
Media precision:  0.7128659698427517
Media recall:  0.9534574468085106
Media f1:  0.8152650976335186

Avaliando base Magner et al com SVM
Media accuracy:  0.6879499217527387
Media precision:  0.7101933387435808
Media recall:  0.8895390070921986
Media f1:  0.7880315864369963


# Comparação no dataset sintético de notícias curtas geradas pelo ChatGPT

In [12]:
print('\n\nAvaliando com base noticias curtas geradas pelo ChatGPT')
df = pd.concat([pd.read_excel('datasets/noticias_negativas_gpt.xlsx'), pd.read_excel('datasets/noticias_positivas_gpt.xlsx')], ignore_index=True)
    
print('\nAvaliando noticias curtas do ChatGPT com VADER')
avalia_algoritmo(df, 'texto', col_titulo=None, col_sentimento='sentimento', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='Vader')

print('\nAvaliando noticias curtas do ChatGPT com MultinomialNB')
avalia_algoritmo(df, 'texto', col_titulo=None, col_sentimento='sentimento', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='MultinomialNB')
    
print('\nAvaliando noticias curtas do ChatGPT com LogisticRegression')
avalia_algoritmo(df, 'texto', col_titulo=None, col_sentimento='sentimento', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='LogisticRegression')
    
print('\nAvaliando noticias curtas do ChatGPT com SVM')
avalia_algoritmo(df, 'texto', col_titulo=None, col_sentimento='sentimento', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='SVC')
    



Avaliando com base noticias curtas geradas pelo ChatGPT

Avaliando noticias curtas do ChatGPT com VADER
Accuracy Score: 0.780
Precision Score: 0.792
Recall Score: 0.760
F1 Score: 0.776

Avaliando noticias curtas do ChatGPT com MultinomialNB
Media accuracy:  0.9400000000000001
Media precision:  0.9102766798418973
Media recall:  0.9800000000000001
Media f1:  0.9428230684044637

Avaliando noticias curtas do ChatGPT com LogisticRegression
Media accuracy:  0.945
Media precision:  0.9337229437229437
Media recall:  0.96
Media f1:  0.9460796926650585

Avaliando noticias curtas do ChatGPT com SVM
Media accuracy:  0.9400000000000001
Media precision:  0.9255411255411256
Media recall:  0.96
Media f1:  0.9411577724799806


# Comparação no dataset sintético de notícias longas geradas pelo ChatGPT

In [13]:
def le_df_chatpgt(arq, sent, label='Notícia'):
    df = pd.read_excel(arq).dropna()

    df.loc[len(df)] = [label + ' X:']

    texts = []
    text = ""

    for s in df[label+' 1:']:
        if ':' in s and s.startswith(label+' ') and len(s) < 15:
            texts.append(text)
            text = ""

        else:
            text = text + s + '\n'


    df = pd.DataFrame({'texto': texts})
    df['sentimento'] = sent
    return df

df = pd.concat([le_df_chatpgt('datasets/noticias_positivas_gpt_longas.xlsx' , 1, 'Notícia'), le_df_chatpgt('datasets/noticias_negativas_gpt_longas.xlsx' , -1, 'Texto')], ignore_index=True)

print('\n\nAvaliando com base noticias longas geradas pelo ChatGPT')

print('\nAvaliando noticias longas do ChatGPT com VADER')
avalia_algoritmo(df, 'texto', col_titulo=None, col_sentimento='sentimento', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='Vader')

print('\nAvaliando noticias longas do ChatGPT com MultinomialNB')
avalia_algoritmo(df, 'texto', col_titulo=None, col_sentimento='sentimento', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='MultinomialNB')
    
print('\nAvaliando noticias longas do ChatGPT LogisticRegression')
avalia_algoritmo(df, 'texto', col_titulo=None, col_sentimento='sentimento', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='LogisticRegression')
    
print('\nAvaliando noticias longas do ChatGPT SVM')
avalia_algoritmo(df, 'texto', col_titulo=None, col_sentimento='sentimento', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='SVC')
    



Avaliando com base noticias longas geradas pelo ChatGPT

Avaliando noticias longas do ChatGPT com VADER
Accuracy Score: 0.950
Precision Score: 0.966
Recall Score: 0.933
F1 Score: 0.949

Avaliando noticias longas do ChatGPT com MultinomialNB
Media accuracy:  1.0
Media precision:  1.0
Media recall:  1.0
Media f1:  1.0

Avaliando noticias longas do ChatGPT LogisticRegression
Media accuracy:  1.0
Media precision:  1.0
Media recall:  1.0
Media f1:  1.0

Avaliando noticias longas do ChatGPT SVM
Media accuracy:  0.9833333333333332
Media precision:  0.9714285714285715
Media recall:  1.0
Media f1:  0.9846153846153847


# Comparação no dataset traduzido de notícias financeiras FinancialPhraseBank
Malo, P., Sinha, A., Korhonen, P., Wallenius, J., & Takala, P. (2014). Good debt or bad debt: Detecting semantic orientations in economic texts. Journal of the Association for Information Science and Technology, 65(4), 782-796.
https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news

In [14]:
import deep_translator
import string
from deep_translator import GoogleTranslator
# traducao do dataset
#gt = GoogleTranslator(source='en', target='pt')
#df = pd.read_csv('FinancialPhraseBank.csv', encoding='ISO-8859-1', header=None, names=['sentiment', 'text'])
#df['translation'] = df['text'].apply(gt.translate)
#df.to_excel('FinancialPhraseBank_translated.xlsx', index=False)
df = pd.read_excel('datasets/FinancialPhraseBank_translated.xlsx')
df = df[df.sentiment != 'neutral']
df['sentiment'] = df.sentiment.apply(lambda x : 1 if x == 'positive' else -1)
df = df.reset_index()

print('\nAvaliando FinancialPhraseBank com VADER')
avalia_algoritmo(df, 'translation', col_titulo=None, col_sentimento='sentiment', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='Vader', resumir=False)

print('\nAvaliando FinancialPhraseBank com MultinomialNB')
avalia_algoritmo(df, 'translation', col_titulo=None, col_sentimento='sentiment', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='MultinomialNB', resumir=False)
    
print('\nAvaliando FinancialPhraseBank LogisticRegression')
avalia_algoritmo(df, 'translation', col_titulo=None, col_sentimento='sentiment', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='LogisticRegression', resumir=False)
    
print('\nAvaliando FinancialPhraseBank SVM')
avalia_algoritmo(df, 'translation', col_titulo=None, col_sentimento='sentiment', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='SVC', resumir=False)
    



Avaliando FinancialPhraseBank com VADER
Accuracy Score: 0.621
Precision Score: 0.778
Recall Score: 0.634
F1 Score: 0.698

Avaliando FinancialPhraseBank com MultinomialNB
Media accuracy:  0.8271528396688238
Media precision:  0.8276841317256357
Media recall:  0.9479045464339582
Media f1:  0.8837167916227313

Avaliando FinancialPhraseBank LogisticRegression
Media accuracy:  0.8357951976853826
Media precision:  0.8194842171760468
Media recall:  0.9787087912087913
Media f1:  0.891997944629804

Avaliando FinancialPhraseBank SVM
Media accuracy:  0.85612172408003
Media precision:  0.853213946888873
Media recall:  0.9574579831932774
Media f1:  0.9022453277696586


In [15]:
len(df)

1967